# Concordancia espacializada

> Notebook organizado para reprodutibilidade. Edite apenas a célula **CONFIGURAÇÕES**.

In [ ]:
from pathlib import Path
import os

# CONFIGURAÇÕES (edite se necessário)
# A pasta raiz do projeto (por padrão, a pasta acima de /notebooks)
ROOT = Path(os.getenv('CLIMBRA_PROJECT_ROOT', Path.cwd().parent)).resolve()
DATA_DIR = ROOT / 'data'
RAW_DIR  = DATA_DIR / '00_raw'
INT_DIR  = DATA_DIR / '01_intermediate'
FINAL_DIR= DATA_DIR / '02_final'
OUT_DIR  = ROOT / 'outputs'
FIG_DIR  = OUT_DIR / 'figures'
TAB_DIR  = OUT_DIR / 'tables'

for d in [RAW_DIR, INT_DIR, FINAL_DIR, FIG_DIR, TAB_DIR]:
    d.mkdir(parents=True, exist_ok=True)


In [ ]:
# ============================================================
# 01_calcular_concordancia_minibacias_SSP585.py
# ============================================================
# Objetivo
#   Ler arquivos de vazão diária (1 arquivo por modelo; 926 colunas = minibacias),
#   calcular (por minibacia e por horizonte):
#     A) TENDÊNCIA (para MK e Theil-Sen):
#        - Construir a série ANUAL como MÉDIA diária do ano (média anual).
#        - Aplicar Mann-Kendall modificado (Hamed-Rao) e inclinação de Theil-Sen
#          sobre essa série anual (por horizonte).
#
#     B) MAGNITUDE ROBUSTA (para ΔQ/Q):
#        - Calcular o "nível" do período base como MEDIANA das vazões anuais
#          no período 1980–2023 (por minibacia).
#        - Calcular o "nível" do horizonte como MEDIANA das vazões anuais
#          dentro do horizonte (por minibacia).
#        - ΔQ/Q (%) = 100 * (Q_h_mediana - Q_base_mediana) / Q_base_mediana
#        (A mediana aqui reduz a influência de anos extremos.)
#
#   Critério de significância por modelo e minibacia:
#     (i) p-valor < 0,05 (MK modificado)
#     (ii) |ΔQ/Q| > 10% (magnitudes robustas por mediana)
#     (iii) sinal (aumento/redução) pelo sinal da inclinação de Theil-Sen
#
#   Além disso, calcula estatísticas do ENSEMBLE (média dos 19 modelos):
#     - Série anual do ensemble = média das séries anuais de cada modelo
#     - Tendência (MK + Sen) nessa série anual média (por horizonte)
#     - ΔQ/Q robusto do ensemble (mediana anual do horizonte vs base)
#
# Saída
#   CSV (formato "long"): sub_bacia × periodo + métricas (português),
#   adequado para JOIN com shapefile e geração de mapas em 4 painéis.
#
# Entradas:
#   FUTURO (2015-2100): E:\RESULTADOS\SSP5_85\QTudo_Fut_Cenário 1
#   HISTÓRICO (1980-2023): E:\RESULTADOS\SSP5_85\Qtudo_Pres
#
# Dependências:
#   pip install pandas numpy pymannkendall scipy
# ============================================================

from __future__ import annotations

import numpy as np
import pandas as pd
from pathlib import Path
import pymannkendall as mk
from scipy.stats import theilslopes

# ------------------------------------------------------------
# CONFIGURAÇÕES (AJUSTE AQUI)
# ------------------------------------------------------------

# Pastas (SSP5-8.5)
PASTA_FUTURO = Path(r"E:\RESULTADOS_AB2\SSP2-45\QTudo_Fut_Cenario 2")
PASTA_PRES   = Path(r"E:\RESULTADOS_AB2\SSP2-45\Qtudo_Pres")

# Saída
PASTA_SAIDA  = Path(r"E:\RESULTADOS_AB2\SSP2-45\_concordancia")
PASTA_SAIDA.mkdir(parents=True, exist_ok=True)
ARQ_SAIDA = PASTA_SAIDA / "concordancia_minibacias_ssp245.csv"

# Arquivos
GLOB_ARQUIVOS = "*.txt"   # ajuste se for "*.TXT"
N_COLS_MINIBACIAS = 926

# Datas completas
DATA_INI_PRES = "1980-01-01"
DATA_FIM_PRES = "2023-12-31"

DATA_INI_FUT  = "2015-01-01"
DATA_FIM_FUT  = "2100-12-31"

# Horizontes (para 4 painéis)
HORIZONTES = {
    "Curto": ("2015-01-01", "2040-12-31"),
    "Médio": ("2041-01-01", "2070-12-31"),
    "Longo": ("2071-01-01", "2100-12-31"),
    "Total": ("2015-01-01", "2100-12-31"),
}

# Critérios
ALFA_PVALOR = 0.05
LIMIAR_DELTA_PCT = 10.0

# Limiares de consenso (mapa 3 cores)
LIMIAR_SEM_CONSENSO = 50.0
LIMIAR_FORTE = 66.0

# Leitura
SEP = r"\s+"
ENGINE_CSV = "python"  # tolerante; se arquivos "limpos", pode ser "c"
DTYPE = "float32"

# ------------------------------------------------------------
# FUNÇÕES AUXILIARES
# ------------------------------------------------------------

def lista_arquivos(pasta: Path, glob_pat: str) -> list[Path]:
    arqs = sorted(pasta.glob(glob_pat))
    if not arqs:
        raise FileNotFoundError(f"Nenhum arquivo encontrado em: {pasta} com padrão {glob_pat}")
    return arqs

def ler_q_diario(arquivo: Path, data_ini: str, data_fim: str, n_cols: int) -> pd.DataFrame:
    """
    Lê arquivo sem cabeçalho com n_cols colunas (minibacias) e linhas diárias.
    Faz ajuste robusto caso exista 1 linha extra (ex.: linha em branco no fim,
    ou arquivo começando 1 dia antes / terminando 1 dia depois).
    """
    df = pd.read_csv(arquivo, header=None, sep=SEP, engine=ENGINE_CSV)

    # Remove linhas totalmente vazias (ex.: linha em branco no final)
    df = df.dropna(how="all")

    if df.shape[1] != n_cols:
        raise ValueError(f"{arquivo.name}: esperado {n_cols} colunas, veio {df.shape[1]}")

    idx = pd.date_range(start=data_ini, end=data_fim, freq="D")
    n_esp = len(idx)
    n_obs = df.shape[0]

    if n_obs != n_esp:
        # Caso típico: 1 linha extra
        if n_obs == n_esp + 1:
            # Heurística: se a primeira linha parece "estranha" (ex.: zeros ou muito diferente),
            # pode ser dia extra no início. Sem datas no arquivo, o mais confiável é:
            # tentar cortar início OU fim e seguir.
            df_tail = df.iloc[1:].reset_index(drop=True)   # remove 1ª linha
            df_head = df.iloc[:-1].reset_index(drop=True)  # remove última linha

            # Escolha padrão: remover última linha (mais comum ser newline/linha extra)
            # Mas se você suspeita que o arquivo começa em 2014-12-31, troque para df_tail.
            df = df_head
            n_obs = df.shape[0]
        else:
            raise ValueError(
                f"{arquivo.name}: esperado {n_esp} linhas (dias), veio {n_obs}. "
                "Verifique se há cabeçalho, linhas extras ou datas ausentes."
            )

    df.index = idx
    df.columns = [f"minibacia_{i+1:04d}" for i in range(n_cols)]
    return df.astype(DTYPE)

def diario_para_serie_anual_media(df_diario: pd.DataFrame) -> pd.DataFrame:
    """
    Série ANUAL para tendência:
    - Média diária do ano (média anual).
    """
    return df_diario.resample("YS").mean()

def mk_pvalor_hamed_rao(x: np.ndarray) -> float:
    if len(x) < 8:
        return np.nan
    return float(mk.hamed_rao_modification_test(x).p)

def theil_sen_inclinacao(x: np.ndarray) -> float:
    """
    Inclinação Theil-Sen por passo (aqui: anual).
    """
    if len(x) < 2:
        return np.nan
    slope, _, _, _ = theilslopes(x, np.arange(len(x)))
    return float(slope)

def sinal_tendencia(slope: float) -> int:
    if not np.isfinite(slope) or slope == 0:
        return 0
    return 1 if slope > 0 else -1

def aplicar_criterios(delta_pct: float, pvalor: float, slope: float) -> int:
    """
    Aplica os 3 critérios:
      p < 0.05, |Δ| > 10%, sinal pelo slope
    Retorna: +1 (aumento), -1 (redução), 0 (não significativo)
    """
    if not (np.isfinite(delta_pct) and np.isfinite(pvalor) and np.isfinite(slope)):
        return 0
    if pvalor >= ALFA_PVALOR:
        return 0
    if abs(delta_pct) <= LIMIAR_DELTA_PCT:
        return 0
    return sinal_tendencia(slope)

def classificar_consenso(agree_pct: float) -> str:
    if not np.isfinite(agree_pct):
        return "sem_consenso"
    if agree_pct < LIMIAR_SEM_CONSENSO:
        return "sem_consenso"
    if agree_pct < LIMIAR_FORTE:
        return "consenso_moderado"
    return "consenso_forte"

def classe_sinal_texto(n_pos: int, n_neg: int, agree_pct: float) -> str:
    if (n_pos + n_neg) == 0:
        return "neutro sem consenso"
    if n_pos == n_neg:
        return "neutro sem consenso"
    dom = "aumento" if n_pos > n_neg else "redução"
    cls = classificar_consenso(agree_pct)
    if cls == "sem_consenso":
        return f"{dom} sem consenso"
    return dom

def tendencia_ensemble_texto(slope: float, pvalor: float) -> str:
    if not (np.isfinite(slope) and np.isfinite(pvalor)):
        return "sem tendência"
    if pvalor >= ALFA_PVALOR or slope == 0:
        return "sem tendência"
    return "tendência de aumento" if slope > 0 else "tendência de redução"

def sinal_texto(slope: float) -> str:
    s = sinal_tendencia(slope)
    if s > 0:
        return "positivo"
    if s < 0:
        return "negativo"
    return "neutro"

def mediana_anual_periodo(df_anual: pd.DataFrame, y0: int, y1: int) -> pd.Series:
    """
    Retorna a MEDIANA das vazões anuais (média anual) dentro do período [y0, y1],
    por minibacia.
    """
    sub = df_anual[(df_anual.index.year >= y0) & (df_anual.index.year <= y1)]
    return sub.median(axis=0)

# ------------------------------------------------------------
# PIPELINE PRINCIPAL
# ------------------------------------------------------------

def main() -> None:
    # 1) Listar arquivos
    arqs_fut = lista_arquivos(PASTA_FUTURO, GLOB_ARQUIVOS)
    arqs_pres = lista_arquivos(PASTA_PRES, GLOB_ARQUIVOS)

    # 2) HISTÓRICO: construir baseline robusto por mediana anual (1980–2023)
    #    Etapas:
    #      - diário -> série anual média (para consistência com tendência)
    #      - nível do período base = mediana das vazões anuais
    #      - baseline multimodelo (se houver vários arquivos) = média dos baselines por modelo
    print(f"[INFO] Arquivos histórico (PRES): {len(arqs_pres)}")
    base_por_modelo = []

    for fp in arqs_pres:
        q_pres_d = ler_q_diario(fp, DATA_INI_PRES, DATA_FIM_PRES, N_COLS_MINIBACIAS)
        q_pres_y = diario_para_serie_anual_media(q_pres_d)  # série anual (média)
        q_base_med = q_pres_y.median(axis=0)                # nível robusto do período base (mediana dos anos)
        base_por_modelo.append(q_base_med)

    if len(base_por_modelo) == 1:
        q_base = base_por_modelo[0]
    else:
        # baseline multimodelo: média entre modelos dos níveis base (robustos por mediana anual)
        q_base = pd.concat(base_por_modelo, axis=1).mean(axis=1)

    minibacias = q_base.index.tolist()
    n_minis = len(minibacias)

    # 3) Preparar acumuladores para concordância e ensemble
    anos_fut = pd.date_range(start=DATA_INI_FUT, end=DATA_FIM_FUT, freq="YS")  # 2015..2100
    n_anos = len(anos_fut)

    # Ensemble anual: soma das séries anuais (média anual) dos modelos, depois divide por N
    ensemble_anual_soma = np.zeros((n_anos, n_minis), dtype=np.float64)

    contagens = {
        nome: {
            "n_pos": np.zeros(n_minis, dtype=np.int32),
            "n_neg": np.zeros(n_minis, dtype=np.int32),
        }
        for nome in HORIZONTES.keys()
    }

    # 4) FUTURO: loop por modelo
    print(f"[INFO] Arquivos futuro (FUT): {len(arqs_fut)}")
    for fp in arqs_fut:
        print(f"[INFO] Processando modelo: {fp.name}")

        q_fut_d = ler_q_diario(fp, DATA_INI_FUT, DATA_FIM_FUT, N_COLS_MINIBACIAS)
        q_fut_y = diario_para_serie_anual_media(q_fut_d)  # série anual (média)

        # alinhar eixo anual
        q_fut_y = q_fut_y.reindex(anos_fut)
        if q_fut_y.isna().any().any():
            raise ValueError(f"Há NaNs após reindex anual para {fp.name}. Verifique integridade das datas.")

        # acumula ensemble anual
        ensemble_anual_soma += q_fut_y.to_numpy(dtype=np.float64)

        # Avaliação por horizonte
        for nome_h, (ini, fim) in HORIZONTES.items():
            y0 = pd.to_datetime(ini).year
            y1 = pd.to_datetime(fim).year

            h = q_fut_y[(q_fut_y.index.year >= y0) & (q_fut_y.index.year <= y1)]
            if len(h) < 8:
                continue

            # ΔQ/Q robusto: mediana anual do horizonte vs mediana anual do base
            q_h_med = h.median(axis=0)  # Series (mediana dos anos do horizonte)
            delta_pct = 100.0 * (q_h_med - q_base) / q_base

            # tendência: MK + Sen na série anual (média anual)
            h_np = h.to_numpy(dtype=np.float64)
            for j in range(n_minis):
                x = h_np[:, j]
                if not np.all(np.isfinite(x)):
                    continue

                p = mk_pvalor_hamed_rao(x)
                slope = theil_sen_inclinacao(x)
                sgn = aplicar_criterios(float(delta_pct.iloc[j]), p, slope)

                if sgn == 1:
                    contagens[nome_h]["n_pos"][j] += 1
                elif sgn == -1:
                    contagens[nome_h]["n_neg"][j] += 1

    # 5) Ensemble anual final
    n_modelos_fut = len(arqs_fut)
    if n_modelos_fut == 0:
        raise RuntimeError("Nenhum arquivo futuro encontrado.")
    ensemble_anual = ensemble_anual_soma / float(n_modelos_fut)

    # 6) Montar tabela final (minibacia × período)
    linhas = []

    for nome_h, (ini, fim) in HORIZONTES.items():
        n_pos = contagens[nome_h]["n_pos"]
        n_neg = contagens[nome_h]["n_neg"]

        agree_n = np.maximum(n_pos, n_neg)
        agree_pct = 100.0 * agree_n / float(n_modelos_fut)

        # Recorte do ensemble no horizonte
        y0 = pd.to_datetime(ini).year
        y1 = pd.to_datetime(fim).year
        mask_ens = (anos_fut.year >= y0) & (anos_fut.year <= y1)
        ens_h = ensemble_anual[mask_ens, :]

        # ΔQ/Q robusto do ensemble: mediana anual do horizonte vs base (q_base)
        ens_h_med = np.nanmedian(ens_h, axis=0)
        ens_delta_pct = 100.0 * (ens_h_med - q_base.to_numpy(dtype=np.float64)) / q_base.to_numpy(dtype=np.float64)

        # Tendência do ensemble: MK + Sen na série anual do ensemble
        for j, mini in enumerate(minibacias):
            x = ens_h[:, j]
            if not np.all(np.isfinite(x)) or len(x) < 8:
                ens_p = np.nan
                ens_slope = np.nan
            else:
                ens_p = mk_pvalor_hamed_rao(x)
                ens_slope = theil_sen_inclinacao(x)

            pct = float(agree_pct[j])
            classe_consenso = classificar_consenso(pct)
            texto_classe_sinal = classe_sinal_texto(int(n_pos[j]), int(n_neg[j]), pct)

            ens_trend_txt = tendencia_ensemble_texto(ens_slope, ens_p)
            ens_sinal_txt = sinal_texto(ens_slope)

            linhas.append({
                "sub_bacia": mini.replace("minibacia_", ""),
                "periodo": nome_h,

                "modelos_total": int(n_modelos_fut),
                "modelos_significativos": int(n_pos[j] + n_neg[j]),
                "positivos": int(n_pos[j]),
                "negativos": int(n_neg[j]),
                "concordancia_pct": round(pct, 1),

                "classe_consenso": classe_consenso,
                "classe_sinal": texto_classe_sinal,

                "ensemble_tendencia": ens_trend_txt,
                "ensemble_p": None if not np.isfinite(ens_p) else float(ens_p),
                "ensemble_slope": None if not np.isfinite(ens_slope) else float(ens_slope),
                "ensemble_var_relativa_pct": None if not np.isfinite(ens_delta_pct[j]) else float(ens_delta_pct[j]),
                "ensemble_significativo": bool(
                    np.isfinite(ens_p)
                    and (ens_p < ALFA_PVALOR)
                    and (abs(ens_delta_pct[j]) > LIMIAR_DELTA_PCT)
                ),
                "ensemble_sinal": ens_sinal_txt,
            })

    df_out = pd.DataFrame(linhas)

    # Ordenação
    df_out["sub_bacia"] = df_out["sub_bacia"].astype(int)
    ordem_periodos = pd.CategoricalDtype(["Curto", "Médio", "Longo", "Total"], ordered=True)
    df_out["periodo"] = df_out["periodo"].astype(ordem_periodos)
    df_out = df_out.sort_values(["sub_bacia", "periodo"]).reset_index(drop=True)

    # Exporta
    df_out.to_csv(ARQ_SAIDA, index=False, encoding="utf-8-sig")
    print(f"[OK] Tabela exportada em: {ARQ_SAIDA}")

if __name__ == "__main__":
    main()

In [ ]:
"""
Script: gera_shapefiles_concordancia_e_mapas_4paineis.py

Descrição geral
---------------
Este script faz duas etapas principais, em sequência:

1) Geração de shapefiles temáticos de CONCORDÂNCIA (classe de consenso)
   - Lê um CSV com resultados (minibacia × período) contendo:
       sub_bacia, periodo, concordancia_pct, classe_consenso, ...
   - Cruza esses dados com o shapefile das minibacias (minis_mgb.shp).
   - Para cada período (Curto/Médio/Longo/Total), grava um shapefile:
       minis_concord_{periodo}.shp
   - IMPORTANTE: Shapefile limita nomes de campos a 10 caracteres.
     Por isso, o script renomeia colunas para nomes curtos antes de salvar.

2) Geração de mapa 2×2 (4 painéis) para o cenário
   - Lê os shapefiles gerados na etapa (1).
   - Plota 4 painéis: Curto, Médio, Longo e Total
   - Simbologia em 3 cores (sem_consenso / consenso_moderado / consenso_forte)
   - Sobrepõe:
       • contorno das sub-bacias (Subbacias.shp)
       • pontos das cidades de Curitiba e União da Vitória
       • rótulos com halo

Uso esperado na pipeline
------------------------
1. Ajustar os CAMINHOS e o campo ID da minibacia (ID_FIELD).
2. Executar o script.
3. Usar a figura final (painéis 2×2) diretamente na dissertação.
"""

# ===================================================
# IMPORTAÇÕES
# ===================================================
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import patheffects
from pathlib import Path

# ===================================================
# 0. CONFIGURAÇÕES – AJUSTAR AQUI
# ===================================================

# Pasta base do cenário
base_dir = Path(r"E:\RESULTADOS_AB2\SSP2-45")  # AJUSTE SE NECESSÁRIO

# CSV de concordância (gerado no script anterior)
# Deve conter: sub_bacia, periodo, concordancia_pct, classe_consenso, ...
csv_concord = base_dir / "_concordancia" / "concordancia_minibacias_ssp245.csv"

# Shapefile das minibacias MGB
shp_minis = Path(r"E:\IGUAÇU_OTTO\6_Calibração\minis_mgb.shp")
ID_FIELD  = "ID_Mini"  # campo de ID da minibacia no shapefile

# Pasta onde serão salvos os shapefiles temáticos
shapes_dir = base_dir / "shapes_concordancia"
shapes_dir.mkdir(exist_ok=True)

# Pasta onde serão salvos os mapas finais (4 painéis)
fig_out_dir = base_dir / "Mapas_4paineis_Concordancia_mediana"
fig_out_dir.mkdir(exist_ok=True)

# Shapefile das sub-bacias (para contorno)
shp_sub = Path(r"E:\IGUAÇU_OTTO\Shp\Subbacias.shp")

# Shapefile das cidades (contendo Curitiba e União da Vitória)
shp_cidades = Path(
    r"G:\Meu Drive\2_MESTRADO\1_Dissertação\Figuras\20250516_SHAPES_FIGURA\GEOFT_CIDADE_2016.shp"
)
CAMPO_NOME_CIDADE = "CID_NM"

# Períodos (ordem fixa para painel)
periodos_ordem = ["Curto", "Médio", "Longo", "Total"]

# Cores (3 classes) – ajuste se desejar
CORES_CONSENSO = {
    "sem_consenso":       "#d9d9d9",  # cinza claro
    "consenso_moderado":  "#9ecae1",  # azul claro
    "consenso_forte":     "#3182bd",  # azul escuro
}

# ===================================================
# 1. ETAPA 1 – GERAR SHAPEFILES TEMÁTICOS (CONCORDÂNCIA)
# ===================================================

print("\n=== ETAPA 1: Gerando shapefiles temáticos de concordância por período ===\n")

# 1.1 Ler CSV
df = pd.read_csv(csv_concord)

# Colunas obrigatórias
req_cols = {"sub_bacia", "periodo", "classe_consenso", "concordancia_pct"}
faltando = req_cols - set(df.columns)
if faltando:
    raise ValueError(f"CSV não contém colunas obrigatórias: {faltando}")

# 1.2 Padronizar tipos
df["sub_bacia"] = df["sub_bacia"].astype(int)
df["periodo"] = df["periodo"].astype(str)

print("Períodos encontrados no CSV:", sorted(df["periodo"].unique()))
print("Minibacias no CSV:", df["sub_bacia"].nunique())

# 1.3 Ler shapefile minibacias
minis_gdf = gpd.read_file(shp_minis)
minis_gdf[ID_FIELD] = minis_gdf[ID_FIELD].astype(int)

# 1.4 Função para “limpar” o rótulo do período para nome de arquivo
def periodo_para_nome(per: str) -> str:
    return (
        per.lower()
        .replace(" ", "_")
        .replace("é", "e")
        .replace("í", "i")
        .replace("ó", "o")
        .replace("ã", "a")
        .replace("ç", "c")
    )

# 1.5 Renomear colunas para compatibilidade com Shapefile (<= 10 caracteres)
REN_SHAPE = {
    "concordancia_pct": "conc_pct",   # 8
    "classe_consenso":  "cls_cons",   # 8
    "classe_sinal":     "cls_sinal",  # 9
    "ensemble_tendencia": "ens_trend",  # 9
    "ensemble_var_relativa_pct": "ens_var",  # 7
    "ensemble_significativo": "ens_sig",     # 7
    "ensemble_sinal": "ens_sinal",           # 9
    "ensemble_slope": "ens_slope",           # 9
    "ensemble_p": "ens_p",                   # 5
}

# 1.6 Gerar shapefile por período
for per in periodos_ordem:
    df_p = df[df["periodo"] == per].copy()

    # Seleciona colunas úteis (se existirem)
    cols_base = ["sub_bacia", "concordancia_pct", "classe_consenso"]
    cols_opc = [
        "classe_sinal",
        "ensemble_tendencia",
        "ensemble_p",
        "ensemble_slope",
        "ensemble_var_relativa_pct",
        "ensemble_significativo",
        "ensemble_sinal",
    ]
    cols = cols_base + [c for c in cols_opc if c in df_p.columns]
    df_p = df_p[cols].copy()

    # Merge com minibacias
    gdf_p = minis_gdf.merge(df_p, left_on=ID_FIELD, right_on="sub_bacia", how="left")

    # Renomeia colunas para o Shapefile (limite 10 caracteres)
    gdf_p = gdf_p.rename(columns={k: v for k, v in REN_SHAPE.items() if k in gdf_p.columns})

    # Salvar
    per_limpo = periodo_para_nome(per)
    shp_out = shapes_dir / f"minis_concord_{per_limpo}.shp"
    gdf_p.to_file(shp_out)
    print(f"Shapefile salvo: {shp_out.name}")

print("\n✔ ETAPA 1 concluída: shapefiles gerados em:", shapes_dir)

# ===================================================
# 2. ETAPA 2 – GERAR MAPA 2×2 (4 PAINÉIS)
# ===================================================

print("\n=== ETAPA 2: Gerando mapa 4 painéis (Curto/Médio/Longo/Total) ===\n")

# Campo de classe no shapefile (renomeado)
CAMPO_CLASSE = "cls_cons"

# 2.1 Ler shapefiles gerados
shps = {}
for per in periodos_ordem:
    per_limpo = periodo_para_nome(per)
    shp_path = shapes_dir / f"minis_concord_{per_limpo}.shp"
    if not shp_path.exists():
        raise FileNotFoundError(f"Não encontrei shapefile do período {per}: {shp_path}")
    shps[per] = shp_path

gdfs = {per: gpd.read_file(path) for per, path in shps.items()}

# Verificação: garantir que CAMPO_CLASSE existe
cols_check = list(gdfs[periodos_ordem[0]].columns)
if CAMPO_CLASSE not in cols_check:
    raise KeyError(
        f"Campo '{CAMPO_CLASSE}' não encontrado no shapefile. Colunas disponíveis: {cols_check}\n"
        "Verifique REN_SHAPE e/ou se o Shapefile truncou nomes de forma diferente."
    )

# CRS
crs_minis = list(gdfs.values())[0].crs

# 2.2 Sub-bacias e cidades
gdf_sub = gpd.read_file(shp_sub).to_crs(crs_minis)
gdf_cid = gpd.read_file(shp_cidades).to_crs(crs_minis)

mask_cur = gdf_cid[CAMPO_NOME_CIDADE].str.contains(r"^curitiba$", case=False, na=False, regex=True)
mask_un  = gdf_cid[CAMPO_NOME_CIDADE].str.contains("uni[aã]o da vit", case=False, na=False, regex=True)
cidades_sel = gdf_cid[mask_cur | mask_un].copy()

# Bounds (usar do primeiro)
xmin, ymin, xmax, ymax = list(gdfs.values())[0].total_bounds
dx = (xmax - xmin) * 0.01
dy = (ymax - ymin) * 0.01

# 2.3 Figura 2×2 + legenda
fig = plt.figure(figsize=(14, 10), dpi=300)
gs = gridspec.GridSpec(2, 3, width_ratios=[1, 1, 0.25], wspace=0.02, hspace=0.05)

ax_11 = fig.add_subplot(gs[0, 0])
ax_12 = fig.add_subplot(gs[0, 1])
ax_21 = fig.add_subplot(gs[1, 0])
ax_22 = fig.add_subplot(gs[1, 1])
ax_leg = fig.add_subplot(gs[:, 2])

axes = [ax_11, ax_12, ax_21, ax_22]
# Ordem lógica (continua igual)
painel_ordem = ["Curto", "Médio", "Longo", "Total"]

# Rótulos de exibição
rotulos_paineis = {
    "Curto": "Horizonte 2015–2040",
    "Médio": "Horizonte 2041–2070",
    "Longo": "Horizonte 2071–2100",
    "Total": "Horizonte Total (2015–2100)",
}

for ax, per in zip(axes, painel_ordem):
    gdf = gdfs[per]

    # Plot por categoria (3 cores fixas)
    for classe, cor in CORES_CONSENSO.items():
        sel = gdf[gdf[CAMPO_CLASSE] == classe]
        if len(sel) == 0:
            continue
        sel.plot(ax=ax, color=cor, edgecolor="black", linewidth=0.05)

    # Contorno sub-bacias
    gdf_sub.boundary.plot(ax=ax, edgecolor="grey", linewidth=1.2, zorder=3)

    # Cidades + rótulos
    if not cidades_sel.empty:
        cidades_sel.plot(ax=ax, marker="^", color="black", markersize=35, zorder=4, linewidth=0)

        for _, row in cidades_sel.iterrows():
            x = row.geometry.x
            y = row.geometry.y
            nome = row[CAMPO_NOME_CIDADE]

            txt = ax.text(
                x + dx, y + dy, nome,
                fontsize=8.5, color="white",
                ha="left", va="bottom", zorder=5
            )
            txt.set_path_effects([
                patheffects.Stroke(linewidth=1.4, foreground="black"),
                patheffects.Normal()
            ])

    ax.set_title(rotulos_paineis[per], fontsize=11)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_aspect("equal")
    ax.set_axis_off()

# 2.4 Legenda customizada (3 classes)
# 2.4 Legenda customizada (3 classes) – CONCORDÂNCIA
ax_leg.axis("off")

ITENS_LEGENDA = [
    ("sem_consenso", "Sem consenso (<50%)"),
    ("consenso_moderado", "Consenso moderado (50–66%)"),
    ("consenso_forte", "Consenso forte (>66%)"),
]

# Parâmetros de layout (ajuste fino aqui se quiser)
n = len(ITENS_LEGENDA)
dy_leg = 0.08          # espaçamento vertical entre itens
y_center = 0.45        # centro vertical do bloco de itens (0 a 1)

# Centro do bloco de itens
y0 = y_center + (n - 1) * dy_leg / 2

# Título da legenda (centralizado em relação ao bloco)
ax_leg.text(
    0.5, y0 + 0.08,
    "Grau de concordância",
    transform=ax_leg.transAxes,
    fontsize=12,
    fontweight="bold",
    ha="center",
    va="bottom"
)

# Itens (caixa + texto)
for i, (key, label) in enumerate(ITENS_LEGENDA):
    y = y0 - i * dy_leg

    # Caixa de cor
    ax_leg.add_patch(
        plt.Rectangle(
            (0.14, y - 0.03), 0.12, 0.06,
            color=CORES_CONSENSO[key],
            transform=ax_leg.transAxes,
            ec="black",
            lw=0.4
        )
    )

    # Texto
    ax_leg.text(
        0.30, y,
        label,
        transform=ax_leg.transAxes,
        fontsize=10,
        va="center",
        ha="left"
    )

# Título geral
fig.suptitle(
    "Grau de concordância entre modelos – Vazão média anual (SSP2-4.5)",
    fontsize=14, weight="bold", y=0.98
)

# Salvar
fig.tight_layout(rect=[0.03, 0.03, 0.98, 0.95])
fig_path = fig_out_dir / "MAPA_4PAINEIS_CONCORDANCIA_SSP245.png"
fig.savefig(fig_path, dpi=300)
plt.close(fig)

print(f"\n✔ Figura salva: {fig_path}")
print("\n✨ ETAPA 2 FINALIZADA – Mapa 4 painéis gerado com sucesso.")

In [ ]:
"""
Script: gera_shapefiles_ensemble_e_mapas_4paineis.py

Objetivo
--------
Gerar mapas 2×2 (4 painéis: Curto/Médio/Longo/Total) mostrando APENAS a
significância do ENSEMBLE (média dos modelos) por minibacia, com base em 3 critérios:

  (i)  ensemble_p < 0,05   (Mann-Kendall modificado aplicado à série anual do ensemble)
  (ii) |ensemble_var_relativa_pct| > 10%
  (iii) sinal da tendência via ensemble_slope (Theil-Sen)

Classificação (3 classes)
-------------------------
- "sem_significancia"       (não atende aos 3 critérios)
- "aumento_significativo"   (atende e slope > 0)
- "reducao_significativa"   (atende e slope < 0)

Entradas
--------
- CSV de concordância já calculado (contém colunas do ensemble):
    sub_bacia, periodo, ensemble_p, ensemble_slope, ensemble_var_relativa_pct
- Shapefile das minibacias (minis_mgb.shp)
- Shapefile das sub-bacias (contorno)
- Shapefile das cidades (Curitiba e União da Vitória)

Saídas
------
- Shapefiles temáticos por período (4):
    minis_enssig_curto.shp, minis_enssig_medio.shp, minis_enssig_longo.shp, minis_enssig_total.shp
- Figura final 2×2:
    MAPA_4PAINEIS_ENSEMBLE_SIGNIFICANCIA_SSP245.png

Observação importante (Shapefile)
---------------------------------
Shapefile limita nomes de campos a 10 caracteres. O script renomeia colunas
antes de salvar (ex.: "ensemble_p" -> "ens_p", "classe_ensemble" -> "cls_ens").
"""

# ===================================================
# IMPORTAÇÕES
# ===================================================
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import patheffects
from pathlib import Path
import numpy as np

# ===================================================
# 0. CONFIGURAÇÕES – AJUSTAR AQUI
# ===================================================

# Pasta base do cenário
base_dir = Path(r"E:\RESULTADOS_AB2\SSP2-45")  # AJUSTE SE NECESSÁRIO

# CSV (gerado no cálculo anterior) – precisa ter colunas do ensemble
csv_in = base_dir / "_concordancia" / "concordancia_minibacias_ssp245.csv"

# Shapefile das minibacias
shp_minis = Path(r"E:\IGUAÇU_OTTO\6_Calibração\minis_mgb.shp")
ID_FIELD  = "ID_Mini"  # campo de ID da minibacia no shapefile

# Sub-bacias (contorno)
shp_sub = Path(r"E:\IGUAÇU_OTTO\Shp\Subbacias.shp")

# Cidades (Curitiba e União da Vitória)
shp_cidades = Path(
    r"G:\Meu Drive\2_MESTRADO\1_Dissertação\Figuras\20250516_SHAPES_FIGURA\GEOFT_CIDADE_2016.shp"
)
CAMPO_NOME_CIDADE = "CID_NM"

# Saídas
shapes_dir = base_dir / "shapes_ensemble_signif"
shapes_dir.mkdir(exist_ok=True)

fig_out_dir = base_dir / "Mapas_4paineis_EnsembleSignif"
fig_out_dir.mkdir(exist_ok=True)

fig_path = fig_out_dir / "MAPA_4PAINEIS_ENSEMBLE_SIGNIFICANCIA_SSP245.png"

# Períodos (ordem fixa)
periodos_ordem = ["Curto", "Médio", "Longo", "Total"]

# Critérios
ALFA_P = 0.05
LIMIAR_DELTA = 10.0  # |var_rel| > 10%

# Cores (3 classes)
CORES = {
    "sem_significancia":      "#d9d9d9",  # cinza
    "aumento_significativo":  "#3182bd",  # azul
    "reducao_significativa":  "#de2d26",  # vermelho
}

# Legenda
LEGENDA = [
    ("sem_significancia", "Sem significância"),
    ("aumento_significativo", "Aumento significativo"),
    ("reducao_significativa", "Redução significativa"),
]

# Função para nomes de arquivo
def periodo_para_nome(per: str) -> str:
    return (
        per.lower()
        .replace(" ", "_")
        .replace("é", "e")
        .replace("í", "i")
        .replace("ó", "o")
        .replace("ã", "a")
        .replace("ç", "c")
    )

# ===================================================
# 1. ETAPA 1 – GERAR SHAPEFILES TEMÁTICOS (ENSEMBLE)
# ===================================================

print("\n=== ETAPA 1: Gerando shapefiles temáticos de significância do ensemble ===\n")

df = pd.read_csv(csv_in)

# Verificações
cols_req = {"sub_bacia", "periodo", "ensemble_p", "ensemble_slope", "ensemble_var_relativa_pct"}
faltando = cols_req - set(df.columns)
if faltando:
    raise ValueError(
        f"CSV não contém colunas obrigatórias para o ensemble: {faltando}\n"
        "Verifique se o script de cálculo exportou as colunas do ensemble."
    )

# Padronizar tipos
df["sub_bacia"] = df["sub_bacia"].astype(int)
df["periodo"] = df["periodo"].astype(str)

# Criar classe de significância do ensemble (3 classes)
def classificar_ensemble(row) -> str:
    p = row["ensemble_p"]
    slope = row["ensemble_slope"]
    varp = row["ensemble_var_relativa_pct"]

    # Trata NaN
    if pd.isna(p) or pd.isna(slope) or pd.isna(varp):
        return "sem_significancia"

    atende = (p < ALFA_P) and (abs(varp) > LIMIAR_DELTA)

    if not atende:
        return "sem_significancia"

    if slope > 0:
        return "aumento_significativo"
    elif slope < 0:
        return "reducao_significativa"
    else:
        return "sem_significancia"

df["classe_ensemble"] = df.apply(classificar_ensemble, axis=1)

# Ler shapefile minibacias
minis_gdf = gpd.read_file(shp_minis)
minis_gdf[ID_FIELD] = minis_gdf[ID_FIELD].astype(int)

# Renomear colunas para Shapefile (<=10 caracteres)
REN_SHAPE = {
    "ensemble_p": "ens_p",
    "ensemble_slope": "ens_slope",
    "ensemble_var_relativa_pct": "ens_var",
    "classe_ensemble": "cls_ens",
}

# Gerar shapefile por período
for per in periodos_ordem:
    df_p = df[df["periodo"] == per].copy()

    # Seleciona colunas essenciais
    df_p = df_p[["sub_bacia", "ensemble_p", "ensemble_slope", "ensemble_var_relativa_pct", "classe_ensemble"]].copy()

    # Merge
    gdf_p = minis_gdf.merge(df_p, left_on=ID_FIELD, right_on="sub_bacia", how="left")

    # Renomear
    gdf_p = gdf_p.rename(columns=REN_SHAPE)

    per_limpo = periodo_para_nome(per)
    shp_out = shapes_dir / f"minis_enssig_{per_limpo}.shp"
    gdf_p.to_file(shp_out)

    print(f"Shapefile salvo: {shp_out.name}")

print("\n✔ ETAPA 1 concluída: shapefiles gerados em:", shapes_dir)

# ===================================================
# 2. ETAPA 2 – GERAR MAPA 2×2 (4 PAINÉIS)
# ===================================================

print("\n=== ETAPA 2: Gerando mapa 4 painéis (significância do ensemble) ===\n")

# Caminhos dos shapefiles por período
shps = {}
for per in periodos_ordem:
    per_limpo = periodo_para_nome(per)
    shp_path = shapes_dir / f"minis_enssig_{per_limpo}.shp"
    if not shp_path.exists():
        raise FileNotFoundError(f"Não encontrei shapefile do período {per}: {shp_path}")
    shps[per] = shp_path

gdfs = {per: gpd.read_file(path) for per, path in shps.items()}

CAMPO_CLASSE = "cls_ens"
if CAMPO_CLASSE not in gdfs[periodos_ordem[0]].columns:
    raise KeyError(
        f"Campo '{CAMPO_CLASSE}' não encontrado no shapefile.\n"
        f"Colunas: {list(gdfs[periodos_ordem[0]].columns)}"
    )

crs_minis = gdfs[periodos_ordem[0]].crs

# Sub-bacias e cidades
gdf_sub = gpd.read_file(shp_sub).to_crs(crs_minis)
gdf_cid = gpd.read_file(shp_cidades).to_crs(crs_minis)

mask_cur = gdf_cid[CAMPO_NOME_CIDADE].str.contains(r"^curitiba$", case=False, na=False, regex=True)
mask_un  = gdf_cid[CAMPO_NOME_CIDADE].str.contains("uni[aã]o da vit", case=False, na=False, regex=True)
cidades_sel = gdf_cid[mask_cur | mask_un].copy()

# Bounds
xmin, ymin, xmax, ymax = gdfs[periodos_ordem[0]].total_bounds
dx = (xmax - xmin) * 0.01
dy = (ymax - ymin) * 0.01

# Figura 2×2 + legenda
fig = plt.figure(figsize=(14, 10), dpi=300)
gs = gridspec.GridSpec(2, 3, width_ratios=[1, 1, 0.22], wspace=0.02, hspace=0.05)

ax_11 = fig.add_subplot(gs[0, 0])
ax_12 = fig.add_subplot(gs[0, 1])
ax_21 = fig.add_subplot(gs[1, 0])
ax_22 = fig.add_subplot(gs[1, 1])
ax_leg = fig.add_subplot(gs[:, 2])

axes = [ax_11, ax_12, ax_21, ax_22]
# Ordem lógica (continua igual)
painel_ordem = ["Curto", "Médio", "Longo", "Total"]

# Rótulos de exibição
rotulos_paineis = {
    "Curto": "Horizonte 2015–2040",
    "Médio": "Horizonte 2041–2070",
    "Longo": "Horizonte 2071–2100",
    "Total": "Horizonte Total (2015–2100)",
}

for ax, per in zip(axes, painel_ordem):
    gdf = gdfs[per]

    # Plot por classe
    for classe, cor in CORES.items():
        sel = gdf[gdf[CAMPO_CLASSE] == classe]
        if len(sel) == 0:
            continue
        sel.plot(ax=ax, color=cor, edgecolor="black", linewidth=0.05)

    # Contorno sub-bacias
    gdf_sub.boundary.plot(ax=ax, edgecolor="grey", linewidth=1.2, zorder=3)

    # Cidades + rótulos
    if not cidades_sel.empty:
        cidades_sel.plot(ax=ax, marker="^", color="black", markersize=35, zorder=4, linewidth=0)

        for _, row in cidades_sel.iterrows():
            x = row.geometry.x
            y = row.geometry.y
            nome = row[CAMPO_NOME_CIDADE]

            txt = ax.text(
                x + dx, y + dy, nome,
                fontsize=8.5, color="white",
                ha="left", va="bottom", zorder=5
            )
            txt.set_path_effects([
                patheffects.Stroke(linewidth=1.4, foreground="black"),
                patheffects.Normal()
            ])

    ax.set_title(rotulos_paineis[per], fontsize=11)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_aspect("equal")
    ax.set_axis_off()

# 2.4 Legenda customizada (3 classes)
ax_leg.axis("off")

# Parâmetros de layout
n = len(LEGENDA)
dy_leg = 0.08

# Centro do bloco de itens
y_center = 0.45
y0 = y_center + (n - 1) * dy_leg / 2

# TÍTULO DA LEGENDA (acima dos itens)
ax_leg.text(
    0.5, y0 + 0.08,
    "Ensemble (Significância)",
    transform=ax_leg.transAxes,
    fontsize=12,
    fontweight="bold",
    ha="center",
    va="bottom"
)

# ITENS DA LEGENDA
for i, (key, label) in enumerate(LEGENDA):
    y = y0 - i * dy_leg

    # Caixa de cor
    ax_leg.add_patch(
        plt.Rectangle(
            (0.18, y - 0.03),
            0.12,
            0.06,
            color=CORES[key],
            transform=ax_leg.transAxes,
            ec="black",
            lw=0.4
        )
    )

    # Texto
    ax_leg.text(
        0.36, y,
        label,
        transform=ax_leg.transAxes,
        fontsize=10,
        va="center",
        ha="left"
    )

# Título geral
fig.suptitle(
    "Significância do ensemble – Vazão média anual (SSP2-4.5)",
    fontsize=14, weight="bold", y=0.98
)

fig.tight_layout(rect=[0.01, 0.02, 0.98, 0.95])
fig.savefig(fig_path, dpi=300)
plt.close(fig)

print(f"\n✔ Figura salva: {fig_path}")
print("\n✨ ETAPA 2 FINALIZADA – Mapa 4 painéis gerado com sucesso.")

In [ ]:
"""
Resumo geral da concordância multimodelo (por horizonte) + classes de consenso

CORREÇÕES IMPORTANTES:
1) Remove BOM (ex.: '\ufeff') do cabeçalho do CSV, que fazia 'sub_bacia' virar '\ufeffsub_bacia'.
2) Contabiliza consenso por horizonte contando MINIBACIAS ÚNICAS (ID), não linhas.
3) Usa as classes no padrão do seu arquivo:
   - sem_consenso
   - consenso_moderado
   - consenso_forte

Saídas (na pasta out_dir):
- resumo_por_periodo_e_estatistica.csv
- resumo_por_periodo_geral.csv
- resumo_concordam_vs_nao_por_periodo.csv
- resumo_texto_word.txt
"""

from pathlib import Path
import pandas as pd

# ==========================================================
# CONFIGURAÇÕES
# ==========================================================
csv_path = Path(r"E:\RESULTADOS_AB2\SSP5-85\_concordancia\concordancia_minibacias_ssp585.csv")

out_dir = Path(r"E:\RESULTADOS_AB2\SSP5-85\_concordancia")
out_dir.mkdir(parents=True, exist_ok=True)

ORDEM_PERIODOS = ["Curto", "Médio", "Longo", "Total"]

# ID da minibacia (deixe assim; o script agora corrige BOM automaticamente)
ID_MINIBACIA_COL = "sub_bacia"

# Classes como aparecem no seu arquivo (print)
CLASSES_ESPERADAS = ["sem_consenso", "consenso_moderado", "consenso_forte"]


# ==========================================================
# FUNÇÕES
# ==========================================================
def limpar_nome_coluna(col: str) -> str:
    """
    Remove BOM e caracteres invisíveis do início/fim, padroniza para lowercase.
    """
    if col is None:
        return col
    # remove BOM e caracteres de "zero width" comuns
    return (
        str(col)
        .replace("\ufeff", "")
        .replace("\u200b", "")
        .strip()
        .lower()
    )


def detectar_id_minibacia(df: pd.DataFrame) -> str:
    candidatos = [
        "sub_bacia", "subbacia", "minibacia", "mini", "codigo_mini", "cod_mini",
        "id_mini", "id_minibacia", "bacia_id", "id"
    ]
    for c in candidatos:
        if c in df.columns:
            return c
    raise ValueError(
        "Não encontrei uma coluna de ID da minibacia automaticamente.\n"
        "Defina ID_MINIBACIA_COL com o nome correto (ex.: 'sub_bacia' ou 'codigo_mini').\n"
        f"Colunas disponíveis: {list(df.columns)}"
    )


def normaliza_classe_consenso(s: pd.Series) -> pd.Series:
    """
    Normaliza classe_consenso para:
    - sem_consenso
    - consenso_moderado
    - consenso_forte
    """
    x = (
        s.astype(str)
         .str.strip()
         .str.lower()
         .str.replace(" ", "_", regex=False)
         .str.replace("-", "_", regex=False)
    )

    # mapeamentos de variantes comuns
    map_variantes = {
        "semconsenso": "sem_consenso",
        "nao_consenso": "sem_consenso",
        "não_consenso": "sem_consenso",

        "moderado": "consenso_moderado",
        "consenso__moderado": "consenso_moderado",

        "forte": "consenso_forte",
        "consenso__forte": "consenso_forte",
    }
    return x.replace(map_variantes)


def padroniza_colunas(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # limpa BOM/invisíveis + lowercase
    df.columns = [limpar_nome_coluna(c) for c in df.columns]

    ren = {}

    for cand in ["periodo", "período", "horizonte", "horizon"]:
        if cand in df.columns:
            ren[cand] = "periodo"
            break

    for cand in ["estatistica", "estatística", "tipo", "variavel", "variável", "metric"]:
        if cand in df.columns:
            ren[cand] = "estatistica"
            break

    for cand in ["modelos_significativos", "modelos_significativo", "modelos_sig", "gcms_significativos"]:
        if cand in df.columns:
            ren[cand] = "modelos_significativos"
            break

    if "positivos" in df.columns:
        ren["positivos"] = "positivos"
    if "negativos" in df.columns:
        ren["negativos"] = "negativos"

    for cand in ["concordancia_pct", "concordância_pct", "concordancia", "concordância", "agreement_pct"]:
        if cand in df.columns:
            ren[cand] = "concordancia_pct"
            break

    if "classe_consenso" in df.columns:
        ren["classe_consenso"] = "classe_consenso"

    df = df.rename(columns=ren)

    # se não existir estatistica, cria
    if "estatistica" not in df.columns:
        df["estatistica"] = "Geral"

    required = [
        "periodo",
        "estatistica",
        "modelos_significativos",
        "positivos",
        "negativos",
        "concordancia_pct",
        "classe_consenso",
    ]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(
            f"Colunas obrigatórias ausentes no CSV: {missing}\n"
            f"Colunas encontradas: {list(df.columns)}"
        )

    # numéricos
    for c in ["modelos_significativos", "positivos", "negativos", "concordancia_pct"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # strings
    df["periodo"] = df["periodo"].astype(str).str.strip()
    df["estatistica"] = df["estatistica"].astype(str).str.strip()

    # normaliza classe_consenso
    df["classe_consenso"] = normaliza_classe_consenso(df["classe_consenso"])

    return df


def resumo_por_grupo(df: pd.DataFrame, group_cols: list[str], id_col: str) -> pd.DataFrame:
    """
    Resumo por grupo contando MINIBACIAS ÚNICAS (id_col).
    """
    g = df.groupby(group_cols, dropna=False)

    out = g.agg(
        modelos_sig_min=("modelos_significativos", "min"),
        modelos_sig_max=("modelos_significativos", "max"),
        conc_pct_min=("concordancia_pct", "min"),
        conc_pct_max=("concordancia_pct", "max"),
        n_minibacias=(id_col, "nunique"),
    ).reset_index()

    # minibacias com >=1 positivo/negativo (únicas)
    pos = (
        df.loc[df["positivos"].fillna(0) > 0]
          .groupby(group_cols, dropna=False)[id_col]
          .nunique()
          .rename("n_minibacias_pos")
          .reset_index()
    )
    neg = (
        df.loc[df["negativos"].fillna(0) > 0]
          .groupby(group_cols, dropna=False)[id_col]
          .nunique()
          .rename("n_minibacias_neg")
          .reset_index()
    )

    out = out.merge(pos, on=group_cols, how="left").merge(neg, on=group_cols, how="left")
    out["n_minibacias_pos"] = out["n_minibacias_pos"].fillna(0).astype(int)
    out["n_minibacias_neg"] = out["n_minibacias_neg"].fillna(0).astype(int)

    # contagem por classe_consenso (únicas)
    ctab = (
        df.groupby(group_cols + ["classe_consenso"], dropna=False)[id_col]
          .nunique()
          .unstack("classe_consenso", fill_value=0)
          .reset_index()
    )

    for c in CLASSES_ESPERADAS:
        if c not in ctab.columns:
            ctab[c] = 0

    ctab = ctab.rename(
        columns={
            "sem_consenso": "n_sem_consenso",
            "consenso_moderado": "n_consenso_moderado",
            "consenso_forte": "n_consenso_forte",
        }
    )

    out = out.merge(
        ctab[group_cols + ["n_sem_consenso", "n_consenso_moderado", "n_consenso_forte"]],
        on=group_cols,
        how="left",
    )

    # concordam vs não
    out["n_concordam"] = out["n_consenso_moderado"].fillna(0).astype(int) + out["n_consenso_forte"].fillna(0).astype(int)
    out["n_nao_concordam"] = out["n_sem_consenso"].fillna(0).astype(int)

    # faixas como texto
    out["faixa_modelos_significativos"] = (
        out["modelos_sig_min"].astype("Int64").astype(str) + "–" +
        out["modelos_sig_max"].astype("Int64").astype(str)
    )
    out["faixa_concordancia_pct"] = (
        out["conc_pct_min"].round(1).astype(str) + "–" +
        out["conc_pct_max"].round(1).astype(str)
    )

    front = group_cols + [
        "faixa_modelos_significativos",
        "n_minibacias_pos",
        "n_minibacias_neg",
        "faixa_concordancia_pct",
        "n_sem_consenso",
        "n_consenso_moderado",
        "n_consenso_forte",
        "n_concordam",
        "n_nao_concordam",
        "n_minibacias",
        "modelos_sig_min", "modelos_sig_max",
        "conc_pct_min", "conc_pct_max",
    ]
    return out[front]


# ==========================================================
# LEITURA
# ==========================================================
df = pd.read_csv(csv_path, sep=None, engine="python")
df = padroniza_colunas(df)

# define coluna ID, com limpeza de BOM/invisíveis
id_col_cfg = limpar_nome_coluna(ID_MINIBACIA_COL) if ID_MINIBACIA_COL else None
if id_col_cfg:
    if id_col_cfg not in df.columns:
        raise ValueError(
            f"ID_MINIBACIA_COL='{ID_MINIBACIA_COL}' não existe no CSV após limpeza.\n"
            f"Colunas: {list(df.columns)}"
        )
    id_col = id_col_cfg
else:
    id_col = detectar_id_minibacia(df)

print(f"[INFO] Coluna de ID da minibacia usada: {id_col}")

# ==========================================================
# RESUMOS
# ==========================================================
res_periodo_est = resumo_por_grupo(df, ["periodo", "estatistica"], id_col=id_col)
res_periodo_geral = resumo_por_grupo(df, ["periodo"], id_col=id_col)

# ordenar períodos
res_periodo_est["periodo_ord"] = pd.Categorical(res_periodo_est["periodo"], categories=ORDEM_PERIODOS, ordered=True)
res_periodo_est = res_periodo_est.sort_values(["periodo_ord", "estatistica"]).drop(columns=["periodo_ord"])

res_periodo_geral["periodo_ord"] = pd.Categorical(res_periodo_geral["periodo"], categories=ORDEM_PERIODOS, ordered=True)
res_periodo_geral = res_periodo_geral.sort_values("periodo_ord").drop(columns=["periodo_ord"])

# tabela extra: só concordam vs não
res_concordam_vs_nao = res_periodo_geral[["periodo", "n_concordam", "n_nao_concordam", "n_minibacias"]].copy()

# ==========================================================
# SALVAR
# ==========================================================
res_periodo_est.to_csv(out_dir / "resumo_por_periodo_e_estatistica.csv", index=False, encoding="utf-8-sig")
res_periodo_geral.to_csv(out_dir / "resumo_por_periodo_geral.csv", index=False, encoding="utf-8-sig")
res_concordam_vs_nao.to_csv(out_dir / "resumo_concordam_vs_nao_por_periodo.csv", index=False, encoding="utf-8-sig")

print("OK! Tabelas geradas em:", out_dir)

# ==========================================================
# TEXTO PRONTO PARA COLAR NO WORD
# ==========================================================
linhas = []
for _, r in res_periodo_geral.iterrows():
    linhas.append(
        f"No horizonte {str(r['periodo']).lower()}, os modelos significativos variaram de "
        f"{r['modelos_sig_min']:.0f} a {r['modelos_sig_max']:.0f}; "
        f"em {int(r['n_minibacias_pos'])} minibacias houve ao menos um modelo com tendência positiva "
        f"e em {int(r['n_minibacias_neg'])} minibacias houve ao menos um modelo com tendência negativa. "
        f"O percentual de concordância variou de {r['conc_pct_min']:.1f}% a {r['conc_pct_max']:.1f}%. "
        f"Quanto ao consenso: {int(r['n_concordam'])} minibacias com consenso (moderado+forte) e "
        f"{int(r['n_nao_concordam'])} minibacias sem consenso "
        f"(forte={int(r['n_consenso_forte'])}, moderado={int(r['n_consenso_moderado'])}, sem={int(r['n_sem_consenso'])})."
    )

txt_path = out_dir / "resumo_texto_word.txt"
with open(txt_path, "w", encoding="utf-8") as f:
    f.write("\n".join(linhas))

print("Texto pronto salvo em:", txt_path)